In [2]:
!pip install fastai -Uqq
!pip install timm -Uqq

In [ ]:
from fastai.vision.all import *
import timm
import torch

In [ ]:
# Import the necessary libraries
from fastai.vision.all import *

# Define the paths to the train and validation directories
path = Path('data')

# Load the data using the fastai library
dls = ImageDataLoaders.from_folder(path=path, train='train_set', valid='valid_set', 
                                   batch_tfms=aug_transforms(mult=1.5, size=150, min_scale=0.75),
                                   item_tfms=Resize(224), bs=32)

In [ ]:
dls.train.show_batch(max_n=8, nrows=2)

In [ ]:
learn = vision_learner(dls, 'convnext_tiny_in22k', metrics=accuracy).to_fp16()

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(5, base_lr=1e-3, cbs=[ShowGraphCallback()])

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(5,5), dpi=60)

In [ ]:
learn.export("./models/learner.pkl")